# Data Preprocess

##### AVG, ERA: 누적O
##### PCT: 누적X

## 1. 제공된 데이터 합치기
<p>2016년부터 2020녀까지의 팀타자 파일을 하나로 통합하고, 2016년부터 2020녀까지의 팀투수 파일을 하나로 통합한다. 
주최 측에서 제공한 데이터 이후의 2020년도 경기들은 크롤링한 후 기존 파일과 같은 형식으로 처리를 하였다. 
    "../data/merged_pitcher_final.csv"와 "../data/merged_hitter_final.csv"에 저장하였다.  

크롤링한 데이터로 구할 수 없는 column들인 "P2_WHIP_RT"(투수), "CB_WHIP_RT"(투수), "LOB"(타자), "P_AB_CN"(타자), "P_HIT_CN"(타자) 지표는 불가피하게 탈락시켰습니다.  <br>
예측 해야할 기간에 가장 중요한 영향을 미치는 정보가 예측해야 하는 경기와 가장 가까운 기간에 있는 데이터라고 판단하였기에 위와 같은 조취를 취했습니다. 
</p>

In [ ]:
!pip install pandas
!pip install numpy

In [1]:
import pandas as pd
import numpy as np

pitcher_df_2016 = pd.read_csv("../data/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv", encoding="utf-8-sig").drop(["P2_WHIP_RT", "CB_WHIP_RT"], axis = 1)
pitcher_df_2017 = pd.read_csv("../data/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2017.csv", encoding="utf-8-sig").drop(["P2_WHIP_RT", "CB_WHIP_RT"], axis = 1) 
pitcher_df_2018 = pd.read_csv("../data/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2018.csv", encoding="utf-8-sig").drop(["P2_WHIP_RT", "CB_WHIP_RT"], axis = 1) 
pitcher_df_2019 = pd.read_csv("../data/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2019.csv", encoding="utf-8-sig").drop(["P2_WHIP_RT", "CB_WHIP_RT"], axis = 1) 
pitcher_df_2020 = pd.read_csv("../data/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2020.csv", encoding="utf-8-sig").drop(["P2_WHIP_RT", "CB_WHIP_RT"], axis = 1) 
pitcher_new = pd.read_csv("../data/merged_pitcher_final.csv", encoding="utf-8-sig") 
pitcher_df = pd.concat([pitcher_df_2016, pitcher_df_2017, pitcher_df_2018, pitcher_df_2019, pitcher_df_2020, pitcher_new])

hitter_df_2016 = pd.read_csv("../data/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv", encoding="utf-8-sig").drop(["LOB", "P_AB_CN", "P_HIT_CN"], axis = 1) 
hitter_df_2017 = pd.read_csv("../data/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2017.csv", encoding="utf-8-sig").drop(["LOB", "P_AB_CN", "P_HIT_CN"], axis = 1) 
hitter_df_2018 = pd.read_csv("../data/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2018.csv", encoding="utf-8-sig").drop(["LOB", "P_AB_CN", "P_HIT_CN"], axis = 1) 
hitter_df_2019 = pd.read_csv("../data/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2019.csv", encoding="utf-8-sig").drop(["LOB", "P_AB_CN", "P_HIT_CN"], axis = 1) 
hitter_df_2020 = pd.read_csv("../data/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2020.csv", encoding="utf-8-sig").drop(["LOB", "P_AB_CN", "P_HIT_CN"], axis = 1) 
hitter_new = pd.read_csv("../data/merged_hitter_final.csv", encoding="utf-8-sig") 
hitter_df = pd.concat([hitter_df_2016, hitter_df_2017, hitter_df_2018, hitter_df_2019, hitter_df_2020, hitter_new])

C:\Users\dhk13\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.
C:\Users\dhk13\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



## 2. 데이터 전처리하기
### 2-1. T_ID 기준 데이터 전처리

<p>투수 지표와 타자 지표에서 겹치는 column을 구분하기 위해서 타자 지표 앞에는 h_, 투수 지표 앞에는 p_를 붙인 후 파일을 합치는 작업<br>
식별을 쉽게 해주기 위해서 해당 경기의 진행년도 열을 YEAR라는 이름으로 추가하였다. <br> 
GAME ID와 GAME DAY, HEADER 경기 여부 등은는 팀타자, 팀투수 파일에 모두 있기 때문에 중복되지 않게 하였다. 
</p>

In [2]:
pitcher_df=pitcher_df[['G_ID','GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'CG_CK', 'WLS', 'HOLD', 'INN2', 'BF', 'PA', 'AB', 'HIT', 'H2', 'H3',
 'HR', 'SB', 'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'WP', 'BK', 'ERR', 'R', 'ER', 'P_WHIP_RT']]

hitter_df=hitter_df[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'PA', 'AB', 'RBI', 'RUN', 'HIT', 'H2', 'H3', 'HR', 'SB',
 'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'ERR', 'P_HRA_RT']]

In [3]:
# 투수 데이터 전처리
pitcher_df["TB_SC"][pitcher_df["TB_SC"] == "B"] = 1
pitcher_df["TB_SC"][pitcher_df["TB_SC"] == "T"] = 0
# pitcher_df["WLS"][pitcher_df["WLS"] == "W"] = 1
# pitcher_df["WLS"][pitcher_df["WLS"] == "L"] = 0
# pitcher_df["WLS"][pitcher_df["WLS"] == "D"] = -1
p_columns = list(pitcher_df.columns)
front_lst = p_columns[:5]
back_lst = p_columns[5:]
back_lst = ["p_" + column for column in back_lst]
p_columns = front_lst + back_lst
pitcher_df.columns = p_columns

# 타자 데이터 전처리
hitter_df.drop(["G_ID", "GDAY_DS", "T_ID", "VS_T_ID", "TB_SC", "HEADER_NO"], axis=1, inplace=True)
h_columns = list(hitter_df.columns)
h_columns = ["h_" + column for column in h_columns]
hitter_df.columns = h_columns
df = pd.concat([pitcher_df, hitter_df], axis=1)
df.rename(columns={"p_WLS":"WLS"}, inplace=True)


# YEAR 데이터 추가
dates = df["GDAY_DS"].to_list()
year = ["".join(list(str(date))[:4]) for date in dates ]
df["YEAR"] = year

# 인덱스 순서 변경
df_columns = list(df.columns)
df_columns_reindex = df_columns[:4] + [df_columns[7]] + [df_columns[-1]] + df_columns[4:7] + df_columns[8:-1]
df = df[df_columns_reindex] 
df.head()

C:\Users\dhk13\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,G_ID,GDAY_DS,T_ID,VS_T_ID,WLS,YEAR,HEADER_NO,p_TB_SC,p_CG_CK,p_HOLD,...,h_CS,h_SH,h_SF,h_BB,h_IB,h_HP,h_KK,h_GD,h_ERR,h_P_HRA_RT
0,20160401HHLG0,20160401,LG,HH,W,2016,0,1,0.0,0.0,...,1.0,1.0,0.0,4.0,0.0,0.0,11.0,0.0,0.0,0.333333
1,20160401HHLG0,20160401,HH,LG,L,2016,0,0,0.0,0.0,...,0.0,3.0,0.0,3.0,0.0,0.0,10.0,1.0,2.0,0.200000
2,20160401HTNC0,20160401,NC,HT,W,2016,0,1,0.0,0.0,...,0.0,1.0,0.0,5.0,0.0,0.0,9.0,1.0,1.0,0.142857
3,20160401HTNC0,20160401,HT,NC,L,2016,0,0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,1.0,10.0,1.0,0.0,0.100000
4,20160401KTSK0,20160401,SK,KT,L,2016,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,1.0,0.375000


### 2-2. VS_T_ID 기준 데이터 전처리
<p> T_ID 기준으로 경기 경기의 결과(방어율, 타율, 승률)가 상대방의 경기력에 영향을 받는다고 판단하여 VS_T_ID(상대편)의 경기 정보를 추가해주었다. 상대방 또한 같은 이름의 columns를 가지고 있기 때문에 앞에 "VS_"를 붙여서 각 줄에 추가하였다.  
</p>

In [4]:
df_vs = df.copy()
df_vs.drop(["WLS", "HEADER_NO"],axis=1, inplace=True)

merge_col = list(df.columns)[:4] + [list(df.columns)[5]]
df_vs_col = list(df.columns)[7:]
vs_rename_col_dict = dict()

for i in df_vs_col:
    vs_rename_col_dict[i] = 'vs_'+i

vs_rename_col_dict["T_ID"] = 'VS_T_ID'
vs_rename_col_dict["VS_T_ID"] = "T_ID"

df_vs = df_vs.rename(columns = vs_rename_col_dict)

df = pd.merge(df, df_vs, how = 'left', left_on = merge_col, right_on = merge_col)

In [17]:
df

,G_ID,GDAY_DS,T_ID,VS_T_ID,WLS,YEAR,HEADER_NO,p_TB_SC,p_CG_CK,p_HOLD,...,vs_h_CS,vs_h_SH,vs_h_SF,vs_h_BB,vs_h_IB,vs_h_HP,vs_h_KK,vs_h_GD,vs_h_ERR,vs_h_P_HRA_RT
0,20160401HHLG0,20160401,LG,HH,W,2016,0,1,0.0,0.0,...,0.0,3.0,0.0,3.0,0.0,0.0,10.0,1.0,2.0,0.200000
1,20160401HHLG0,20160401,HH,LG,L,2016,0,0,0.0,0.0,...,1.0,1.0,0.0,4.0,0.0,0.0,11.0,0.0,0.0,0.333333
2,20160401HTNC0,20160401,NC,HT,W,2016,0,1,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,1.0,10.0,1.0,0.0,0.100000
3,20160401HTNC0,20160401,HT,NC,L,2016,0,0,0.0,0.0,...,0.0,1.0,0.0,5.0,0.0,0.0,9.0,1.0,1.0,0.142857
4,20160401KTSK0,20160401,SK,KT,L,2016,0,1,0.0,0.0,...,1.0,0.0,1.0,4.0,0.0,1.0,7.0,0.0,0.0,0.285714
5,20160401KTSK0,20160401,KT,SK,W,2016,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,1.0,0.375000
6,20160401LTWO0,20160401,WO,LT,L,2016,0,1,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,6.0,2.0,0.0,0.071429
7,20160401LTWO0,20160401,LT,WO,W,2016,0,0,0.0,4.0,...,2.0,0.0,1.0,5.0,0.0,0.0,6.0,0.0,1.0,0.000000
8,20160401OBSS0,20160401,SS,OB,L,2016,0,1,0.0,0.0,...,1.0,1.0,1.0,3.0,0.0,1.0,3.0,1.0,0.0,0.142857
9,20160401OBSS0,20160401,OB,SS,W,2016,0,0,0.0,1.0,...,1.0,0.0,0.0,2.0,0.0,1.0,6.0,2.0,1.0,0.250000


## 3. 24개 경기로 합치기
<p>저희 SWEEEEEP에서는 각 팀의 지표를 예측할 때, 각 경기별로 예측하기 보다는 남은 경기(24경기)의 결과를 통째로 예측하는 방식을 선택했다. 

해당 방식을 위해서는 경기 기록을 24경기 씩 묶는 방식으로 데이터를 가공하였다.

그러나 데이터를 24경기 씩 묶으면 데이터 셋의 갯수가 적어지기 때문에 아래와 같은 방법으로 data augmentation을 진행하였다. 


이때 train, valid를 미리 나누어 둔 후, train 부분에 대해서만 데이터 늘리기 적용</p>

### 3-1. column 분류
<p>현재 Dataframe에는 T_ID(해당 팀), p_(해당 팀의 투수관련 지표들), h_(해당 팀의 타자관련 지표들), vs_p_(상대팀의 투구 관련 지표들), vs_h_(상대팀의 타자관련 지표들), 경기 정보 Column들로 구성이 되어있다. 

여기서 24경기 씩 묶어 주기 위해서 승패column과 나머지 column을 구분하는 부분이다. 

size_df에는 경기정보와 승패
sum_df에는 경기정보와 경기내의 투수 타자 지표들이 있다. </p>

In [9]:
data_col = list(df.columns)
team = list(df.T_ID.unique())

default_col = ['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', "YEAR"]
size_col = ['WLS']
sum_col = list(df.columns)[6:]

tmp = set(default_col)|set(size_col)|set(sum_col)
size_df = df[default_col + size_col]
sum_df = df[default_col + sum_col]

### 3-2. Size & Sum

In [10]:
# ver='t': VS_T_ID고려안함 & ver='vs': VS_T_ID고려
def make_pct(df, ver='t'):
    col1 = ["T_ID","WLS","YEAR"]
    col2 = ["T_ID","YEAR"]
    if ver=='vs':
        col1.append("VS_T_ID")
        col2.append("VS_T_ID")
        
    tmp = pd.DataFrame({"COUNT":df.groupby(col1).size()}).pivot_table("COUNT",col2,"WLS").reset_index().sort_values(["YEAR","T_ID"]).reset_index(drop=True)
    tmp.fillna(0,inplace= True)

    if "L" not in list(tmp.columns):
        tmp["L"] = 0

    if "W" not in list(tmp.columns):
        tmp["W"] = 0
    
    tmp["PCT"] = tmp['W']/(tmp['W']+tmp['L'])
    return tmp

In [11]:
def make_sum(df, ver='t'):
    col = ["T_ID","YEAR"]
    if ver == 'vs':
        col.append("VS_T_ID")
    
    tmp = df.groupby(col).sum().reset_index()
    return tmp

In [12]:
# ver='c': 누적데이터 & ver='o':순수데이터

def calculate_df(df, valid_year, key, sc='sum', ver='o'):

    df_cal = pd.DataFrame([])

    for y in [2016,2017,2018,2019,2020]:
        print("===",y)
        y_tmp = df[df["YEAR"]==str(y)]
        for t in team:
            print(t)
            
            ttmp_df = y_tmp[y_tmp["T_ID"]==t]
            tmp_size = len(ttmp_df)

            #validset
            if y in valid_year:
                if ver == 'c':
                    test_tmp_df = ttmp_df.iloc[:tmp_size-key,:]
                else:
                    test_tmp_df = ttmp_df.iloc[tmp_size-key:,:]
                    
                if sc == 'size':
                    tmp2 = make_pct(test_tmp_df)
                elif sc == 'sum_ae':
                    tmp2 = make_sum(test_tmp_df,'vs')
                else:
                    tmp2 = make_sum(test_tmp_df)
                tmp2['IDX'] = 777
                tmp2['MERGE_IDX'] = 0

                df_cal = pd.concat([df_cal, tmp2], axis = 0)

            # trainset
            if y in valid_year: # valid_data인 경우->120개 이용
                tmp_df = ttmp_df.iloc[:tmp_size-key,:]
                t_size = len(tmp_df)
            else: # train_data인 경우->전체 데이터이용
                tmp_df = ttmp_df.copy()
                t_size = len(tmp_df)
            
            idx = 0
            m_idx = 1
            while idx < key:
                k_idx = 1
                for k in range((t_size-idx)//key):
                    
                    if ver == 'c':
                        tmp1 = tmp_df.iloc[:key*(k+1)+idx,:]
                    else:
                        tmp1 = tmp_df.iloc[key*k+idx:key*(k+1)+idx,:]
                    
                    if sc == 'size':
                        tmp2 = make_pct(tmp1)
                    elif sc == 'sum_ae':
                        tmp2 = make_sum(tmp1,'vs')
                    else:
                        tmp2 = make_sum(tmp1)
                        
                    tmp2['IDX'] = k_idx 
                    tmp2['MERGE_IDX'] = m_idx

                    k_idx += 1
                    m_idx += 1

                    df_cal = pd.concat([df_cal, tmp2], axis = 0)
                idx += 1
                
    if sc == 'size':
        df_cal.drop(["W","D","L"], axis = 1, inplace = True)
    
    df_cal = df_cal.sort_values(by=["YEAR","T_ID","MERGE_IDX"]).reset_index(drop = True)
    return df_cal

## 4. 추가 데이터 생성

### 4-1. HOME팀 및 상대팀 구성 비율

In [18]:
df_new = pd.concat([df, pd.get_dummies(df.VS_T_ID)], axis = 1)

for i in range(len(df_new)):
    if df_new.loc[i,'T_ID'] == df_new.loc[i,"G_ID"][10:12]:
        df_new.loc[i,'ISHOME'] = 1
    else:
        df_new.loc[i,'ISHOME'] = 0
        
df_new["ISHOME"] = df_new["ISHOME"].astype(np.int64)

home_n_vs_df = df_new[["G_ID","YEAR","T_ID","VS_T_ID","ISHOME"]+team]

### 4-2. 누적 상대타율/방어율

In [19]:
def_col = ['G_ID','T_ID', 'VS_T_ID',"YEAR"]
ae_col = ['p_INN2', 'p_ER', 'h_HIT','h_AB']
ae_df = df[def_col + ae_col]

ae_df

,G_ID,T_ID,VS_T_ID,YEAR,p_INN2,p_ER,h_HIT,h_AB
0,20160401HHLG0,LG,HH,2016,36.0,4.0,9.0,42.0
1,20160401HHLG0,HH,LG,2016,34.0,4.0,13.0,46.0
2,20160401HTNC0,NC,HT,2016,27.0,4.0,9.0,30.0
3,20160401HTNC0,HT,NC,2016,24.0,5.0,8.0,34.0
4,20160401KTSK0,SK,KT,2016,27.0,8.0,10.0,36.0
5,20160401KTSK0,KT,SK,2016,27.0,4.0,11.0,36.0
6,20160401LTWO0,WO,LT,2016,27.0,1.0,6.0,30.0
7,20160401LTWO0,LT,WO,2016,27.0,0.0,11.0,37.0
8,20160401OBSS0,SS,OB,2016,27.0,4.0,8.0,32.0
9,20160401OBSS0,OB,SS,2016,27.0,1.0,8.0,31.0


### 4-3. 데이터 생성

In [ ]:
year = [2016,2017,2018,2019]

Home_VS_lst = []

cteam_AVG_lst = []
cteam_ERA_lst = []

for y in year:
    valid_y = year[:]
    valid_y.remove(y)
    
    # home and vs_count
    cal_h_vs_df = calculate_df(home_n_vs_df, valid_y, 24, sc='sum', ver='o')
    cal_h_vs_df_c = cal_h_vs_df.add_prefix('c_').iloc[:,3:13]
    cal_h_vs_df_c = cal_h_vs_df_c/24
    cal_hvs_df = pd.concat([cal_h_vs_df.iloc[:,[0,1,2,13,14]],cal_h_vs_df_c],axis=1)
    
    Home_VS_lst.append(cal_hvs_df)

    # cumulate vs era & vs avg
    cal_ae_df = calculate_df(ae_df,valid_y ,24, sc='sum_ae', ver='c')

    cal_ae_df["ERA"] = (9 * cal_ae_df["p_ER"]) / (cal_ae_df["p_INN2"] / 3)
    cal_ae_df["AVG"] = (cal_ae_df["h_HIT"] / cal_ae_df["h_AB"])

    cul_t_avg_df = cal_ae_df.drop(["ERA"]+ae_col,axis=1)
    cul_t_era_df = cal_ae_df.drop(["AVG"]+ae_col,axis=1)
    
    cul_avg_df = cul_t_avg_df.pivot_table('AVG', ['T_ID', 'YEAR', 'IDX', 'MERGE_IDX'], 'VS_T_ID').reset_index().sort_values(by =["YEAR","T_ID","MERGE_IDX"]).reset_index(drop = True).fillna(0)
    cul_era_df = cul_t_era_df.pivot_table('ERA', ['T_ID', 'YEAR', 'IDX', 'MERGE_IDX'], 'VS_T_ID').reset_index().sort_values(by =["YEAR","T_ID","MERGE_IDX"]).reset_index(drop = True).fillna(0)

    ca = cul_avg_df.iloc[:,-10:].add_prefix('a_')
    ce = cul_era_df.iloc[:,-10:].add_prefix('e_')
    
    cteam_AVG_lst.append(ca)
    cteam_ERA_lst.append(ce)
    

## 5. 데이터 합치면서 세이버매트릭스 변환

### 5-1. 세이버매트릭스 변환 함수

In [ ]:
# 공통 부분
# T_ID, YEAR, PCT, IDX, MERGE_IDX

# T_ID 팀
# p_CG_CK, p_HOLD, p_INN2, p_BF, p_PA, p_AB, p_HIT, p_H2, p_H3
# p_HR, p_SB, p_CS, p_SH, p_SF, p_BB, p_IB, p_HP, p_KK, p_GD, p_WP, p_BK, p_ERR, p_R, p_ER, p_P_WHIP_RT, p_P2_WHIP_RT
# p_CB_WHIP_RT, h_PA, h_AB, h_RBI, h_RUN, h_HIT, h_H2, h_H3, h_HR, h_SB, h_CS, h_SH, h_SF, h_BB, h_IB, h_HP, h_KK
# h_GD, h_ERR, h_LOB, h_P_HRA_RT, h_P_AB_CN, h_P_HIT_CN

# vs_T_ID 팀 (vs_만 붙이면 같음)

'''
@ Augment를 세이버매트릭스로 변환하기 전 주의사항
1. P_WHIP_RT, P2_WHIP_RT, CB_WHIP_RT, P_HRA_RT를 그대로 누적했는데 그대로 누적해서
사용하면 안되는 값이다.
- 값이 너무 커서 일단 24로 나눠서 경기 당 평균값으로 만들었다
- RC도 값이 커서 24로 나눠서 경기 당 평균값으로 만들었다.

2.변환 이후 정규화

3.이닝 수, 득점, 타수, 실점 기록 추가가 필요할까..? (승률에서는 일단 제외)
'''


def convert_attribute(df, ver ='tv'):
    #column

    new_index = ["T_ID", "YEAR", "PCT", "IDX", "MERGE_IDX",
             "WHIP", "LOB", "ERA", "FIP", "H_9", "K_9", "BB_9", "HR_9",
             "AVG", "oAVG", "SLG", "oSLG", "OBP", "oOBP", "P_WHIP_RT",
             "OPS", "oOPS", "RC", "GPA", "SECA", "TA", "ISO", "oISO", "wOBA", "P_HRA_RT", "DER",
             "vs_WHIP", "vs_LOB", "vs_ERA", "vs_FIP", "vs_H_9", "vs_K_9", "vs_BB_9", "vs_HR_9",
             "vs_AVG", "vs_oAVG", "vs_SLG", "vs_oSLG", "vs_OBP", "vs_oOBP", "vs_P_WHIP_RT",
             "vs_OPS", "vs_oOPS", "vs_RC", "vs_GPA", "vs_SECA", "vs_TA", "vs_ISO", "vs_oISO", "vs_wOBA", "vs_P_HRA_RT", "vs_DER"]
    
    if ver == 'test':
        new_index.remove("IDX")
        new_index.remove("MERGE_IDX")
        
    
    #시작
    new_df = pd.DataFrame(columns = new_index)
    
    # 공통 부분T_ID, YEAR, PCT, IDX, MERGE_IDX
    new_df["T_ID"] = df["T_ID"]
    new_df["YEAR"] = df["YEAR"]
    new_df["PCT"] = df["PCT"]
    
    if ver != 'test':
        new_df["IDX"] = df["IDX"]
        new_df["MERGE_IDX"] = df["MERGE_IDX"]
    
    
    # 자주 사용되는 변환 값
    p_H1 = (df["p_HIT"] - df["p_H2"] - df["p_H3"] - df["p_HR"]) #피 1루타
    h_H1 = (df["h_HIT"] - df["h_H2"] - df["h_H3"] - df["h_HR"]) #1 루타
    RC_A = (df["h_HIT"] + df["h_BB"] - df["h_CS"] + df["h_HP"] - df["h_GD"]) #출루능력
    RC_B = 0.24 * (df["h_BB"] - df["h_IB"] + df["h_HP"]) + (0.62 * df["h_SB"]) + 0.5 * (df["h_SH"] + df["h_SF"]) - 0.03 * df["h_KK"] #진루능력
    RC_C = (df["h_AB"] + df["h_BB"] + df["h_HP"] + df["h_SH"] + df["h_SF"]) #주어진 기회
    
    vs_p_H1 = (df["vs_p_HIT"] - df["vs_p_H2"] - df["vs_p_H3"] - df["vs_p_HR"]) #피 1루타
    vs_h_H1 = (df["vs_h_HIT"] - df["vs_h_H2"] - df["vs_h_H3"] - df["vs_h_HR"]) #1 루타
    vs_RC_A = (df["vs_h_HIT"] + df["vs_h_BB"] - df["vs_h_CS"] + df["vs_h_HP"] - df["vs_h_GD"]) #출루능력
    vs_RC_B = 0.24 * (df["vs_h_BB"] - df["vs_h_IB"] + df["vs_h_HP"]) + (0.62 * df["vs_h_SB"]) + 0.5 * (df["vs_h_SH"] + df["vs_h_SF"]) - 0.03 * df["vs_h_KK"] #진루능력
    vs_RC_C = (df["vs_h_AB"] + df["vs_h_BB"] + df["vs_h_HP"] + df["vs_h_SH"] + df["vs_h_SF"]) #주어진 기회
    #----------------------- T_ID
    new_df["WHIP"] = (df["p_HIT"] + df["p_BB"]) / (df["p_INN2"] / 3)
    new_df["LOB"] = (df["p_HIT"] + df["p_BB"] + df["p_HP"] - df["p_R"]) / (df["p_HIT"] + df["p_BB"] + df["p_HP"] - (1.4 * df["p_HR"]))
    new_df["ERA"] = (9 * df["p_ER"]) / (df["p_INN2"] / 3)
    new_df["FIP"] = ((13 * df["p_HR"]) + (3 * (df["p_BB"] + df["p_HP"] - df["p_IB"])) - (2 * df["p_KK"])) / (df["p_INN2"] / 3)
    new_df["H_9"] = (9 * df["p_HIT"]) / (df["p_INN2"] / 3)
    new_df["K_9"] = (9 * df["p_KK"]) / (df["p_INN2"] / 3)
    new_df["BB_9"] = (9 * df["p_BB"]) / (df["p_INN2"] / 3)
    new_df["HR_9"] = (9 * df["p_HR"]) / (df["p_INN2"] / 3)
    new_df["AVG"] = (df["h_HIT"] / df["h_AB"])
    new_df["oAVG"] = (df["p_HIT"] / (df["p_PA"] - df["p_BB"] - df["p_HP"] - df["p_SF"] - df["p_SB"]))
    new_df["SLG"] = (h_H1 + df["h_H2"] * 2 + df["h_H3"] * 3 + df["h_HR"] * 4) / (df["h_AB"])
    new_df["oSLG"] = (p_H1 + (2 * df["p_H2"]) + (3 * df["p_H3"]) + (4 * df["p_HR"])) / (df["p_PA"] - df["p_BB"] - df["p_HP"] - df["p_SF"] - df["p_SB"])
    new_df["OBP"] = (df["h_HIT"] + df["h_BB"] + df["h_HP"]) / (df["h_AB"] + df["h_BB"] + df["h_HP"] + df["h_SF"])
    new_df["oOBP"] = (df["p_HIT"] + df["p_BB"]) / (df["p_AB"] + df["p_BB"] + df["p_HP"] + df["p_SF"])
    new_df["P_WHIP_RT"] = (df["p_P_WHIP_RT"] / 24) 
    new_df["OPS"] = (new_df["OBP"] + new_df["SLG"])
    new_df["oOPS"] = (new_df["oOBP"] + new_df["oSLG"])
    new_df["RC"] = (((2.4 * RC_C + RC_A) * (3 * RC_C + RC_B)) / (9 * RC_C) - (0.9 * RC_C)) / 24
    new_df["GPA"] = ((1.8 * new_df["OBP"] + new_df["SLG"]) / 4)
    new_df["SECA"] = (((df["h_H2"] * 2) + (df["h_H3"] * 3) + (df["h_HR"] * 4) + df["h_BB"] + (df["h_SB"] - df["h_CS"])) / df["h_AB"])
    new_df["TA"] = ((h_H1 + (df["h_H2"] * 2) + (df["h_H3"] * 3) + (df["h_HR"] * 4) + df["h_HP"] + df["h_BB"] + df["h_SB"]) / (df["h_AB"] - df["h_HIT"] + df["h_CS"] + df["h_GD"]))
    new_df["ISO"] = (new_df["SLG"] - new_df["AVG"])
    new_df["oISO"] = (new_df["oSLG"] - new_df["oAVG"])
    new_df["wOBA"] = ((0.7 * (df["h_BB"] - df["h_IB"]) + (0.73 * df["h_HP"]) + (0.89 * h_H1) + (1.27 * df["h_H2"]) + (1.61 * df["h_H3"]) + (2.07 * df["h_HR"]) + (0.25 * df["h_SB"]) -
                (0.5 * df["h_CS"])) / (df["h_AB"] - df["h_IB"]))
    new_df["P_HRA_RT"] = df["h_P_HRA_RT"] 
    new_df["DER"] = ((df["p_PA"] - df["p_HIT"] - df["p_KK"] - df["p_BB"] - df["p_HP"] - df["p_IB"] - df["p_BK"]) / (df["p_PA"] - df["p_HR"] - df["p_KK"] - df["p_BB"] - df["p_HP"]))
    
    
    #----------------------- vs_T_ID
    new_df["vs_WHIP"] = (df["vs_p_HIT"] + df["vs_p_BB"]) / (df["vs_p_INN2"] / 3)
    new_df["vs_LOB"] = (df["vs_p_HIT"] + df["vs_p_BB"] + df["vs_p_HP"] - df["vs_p_R"]) / (df["vs_p_HIT"] + df["vs_p_BB"] + df["vs_p_HP"] - (1.4 * df["vs_p_HR"]))
    new_df["vs_ERA"] = (9 * df["vs_p_ER"]) / (df["vs_p_INN2"] / 3)
    new_df["vs_FIP"] = ((13 * df["vs_p_HR"]) + (3 * (df["vs_p_BB"] + df["vs_p_HP"] - df["vs_p_IB"])) - (2 * df["vs_p_KK"])) / (df["vs_p_INN2"] / 3)
    new_df["vs_H_9"] = (9 * df["vs_p_HIT"]) / (df["vs_p_INN2"] / 3)
    new_df["vs_K_9"] = (9 * df["vs_p_KK"]) / (df["vs_p_INN2"] / 3)
    new_df["vs_BB_9"] = (9 * df["vs_p_BB"]) / (df["vs_p_INN2"] / 3)
    new_df["vs_HR_9"] = (9 * df["vs_p_HR"]) / (df["vs_p_INN2"] / 3)
    new_df["vs_AVG"] = (df["vs_h_HIT"] / df["vs_h_AB"])
    new_df["vs_oAVG"] = (df["vs_p_HIT"] / (df["vs_p_PA"] - df["vs_p_BB"] - df["vs_p_HP"] - df["vs_p_SF"] - df["vs_p_SB"]))
    new_df["vs_SLG"] = (vs_h_H1 + df["vs_h_H2"] * 2 + df["vs_h_H3"] * 3 + df["vs_h_HR"] * 4) / (df["vs_h_AB"])
    new_df["vs_oSLG"] = (vs_p_H1 + (2 * df["vs_p_H2"]) + (3 * df["vs_p_H3"]) + (4 * df["vs_p_HR"])) / (df["vs_p_PA"] - df["vs_p_BB"] - df["vs_p_HP"] - df["vs_p_SF"] - df["vs_p_SB"])
    new_df["vs_OBP"] = (df["vs_h_HIT"] + df["vs_h_BB"] + df["vs_h_HP"]) / (df["vs_h_AB"] + df["vs_h_BB"] + df["vs_h_HP"] + df["vs_h_SF"])
    new_df["vs_oOBP"] = (df["vs_p_HIT"] + df["vs_p_BB"]) / (df["vs_p_AB"] + df["vs_p_BB"] + df["vs_p_HP"] + df["vs_p_SF"])
    new_df["vs_P_WHIP_RT"] = (df["vs_p_P_WHIP_RT"] / 24) 
    new_df["vs_OPS"] = (new_df["vs_OBP"] + new_df["vs_SLG"])
    new_df["vs_oOPS"] = (new_df["vs_oOBP"] + new_df["vs_oSLG"])
    new_df["vs_RC"] = (((2.4 * vs_RC_C + vs_RC_A) * (3 * vs_RC_C + vs_RC_B)) / (9 * vs_RC_C) - (0.9 * vs_RC_C)) / 24
    new_df["vs_GPA"] = ((1.8 * new_df["vs_OBP"] + new_df["vs_SLG"]) / 4)
    new_df["vs_SECA"] = (((df["vs_h_H2"] * 2) + (df["vs_h_H3"] * 3) + (df["vs_h_HR"] * 4) + df["vs_h_BB"] + (df["vs_h_SB"] - df["vs_h_CS"])) / df["vs_h_AB"])
    new_df["vs_TA"] = ((h_H1 + (df["vs_h_H2"] * 2) + (df["vs_h_H3"] * 3) + (df["vs_h_HR"] * 4) + df["vs_h_HP"] + df["vs_h_BB"] + df["vs_h_SB"]) / (df["vs_h_AB"] - df["vs_h_HIT"] + df["vs_h_CS"] + df["vs_h_GD"]))
    new_df["vs_ISO"] = (new_df["vs_SLG"] - new_df["vs_AVG"])
    new_df["vs_oISO"] = (new_df["vs_oSLG"] - new_df["vs_oAVG"])
    new_df["vs_wOBA"] = ((0.7 * (df["vs_h_BB"] - df["vs_h_IB"]) + (0.73 * df["vs_h_HP"]) + (0.89 * vs_h_H1) + (1.27 * df["vs_h_H2"]) + (1.61 * df["vs_h_H3"]) + (2.07 * df["vs_h_HR"]) + (0.25 * df["vs_h_SB"]) -
                (0.5 * df["vs_h_CS"])) / (df["vs_h_AB"] - df["vs_h_IB"]))
    new_df["vs_P_HRA_RT"] = (df["vs_h_P_HRA_RT"] / 24)
    new_df["vs_DER"] = ((df["vs_p_PA"] - df["vs_p_HIT"] - df["vs_p_KK"] - df["vs_p_BB"] - df["vs_p_HP"] - df["vs_p_IB"] - df["vs_p_BK"]) / (df["vs_p_PA"] - df["vs_p_HR"] - df["vs_p_KK"] - df["vs_p_BB"] - df["vs_p_HP"]))

    
    #new_df.to_csv("convert_temp.csv") 파일로 변환해서 확인해봤습니다.
    return new_df


### 5-2. 데이터 계산 및 합치기

In [ ]:
year = [2016,2017,2018,2019]
orig_data = []
cumul_data = []

for y in year:
    valid_y = year[:]
    valid_y.remove(y)
    
    # 누적X
    size_df_cal1 = calculate_df(size_df, valid_y, 24, 'size', ver='o')
    sum_df_cal1 = calculate_df(sum_df, valid_y, 24, 'sum', ver='o')
    merge_df1 = pd.merge(size_df_cal1, sum_df_cal1, how= 'left', left_on = ['T_ID',"YEAR","MERGE_IDX","IDX"], right_on  = ['T_ID',"YEAR","MERGE_IDX","IDX"])

    conv1 = convert_attribute(merge_df1)
    orig_data.append(conv1)
    
    # 누적O
    size_df_cal2 = calculate_df(size_df, valid_y, 24, 'size', ver='c')
    sum_df_cal2 = calculate_df(sum_df, valid_y, 24, 'sum', ver='c')
    merge_df2 = pd.merge(size_df_cal2, sum_df_cal2, how= 'left', left_on = ['T_ID',"YEAR","MERGE_IDX","IDX"], right_on  = ['T_ID',"YEAR","MERGE_IDX","IDX"])
    
    conv2 = convert_attribute(merge_df2)
    
    cumul_data.append(conv2)

In [ ]:
# for i in range(len(orig_data)):
#     orig_data[i].to_csv("orig_dataset_{}.csv".format(i), index = False)
#     cumul_data[i].to_csv("cumul_dataset_{}.csv".format(i), index = False)

### 5-3. 누적 데이터 + 누적 AVG/ERA 상대 타/방

In [ ]:
new_cumul_data = []

for i in range(len(cumul_data)):
    tmp_new = pd.concat([cumul_data[i],cteam_AVG_lst[i], cteam_ERA_lst[i]],axis=1)
    new_cumul_data.append(tmp_new)

# Train / Valid / Test split

In [ ]:
def shifting(df):
    sel_col = ["T_ID","IDX","MERGE_IDX","PCT","ERA","AVG",'YEAR']
    df_shift = df.loc[:,sel_col]
    shift_col = sorted(list(set(list(df.columns))-set(['T_ID','YEAR'])))
    
    for i in range(1,2):
        for c in shift_col:
            df_shift.loc[:,'shift_{}'.format(c)] = df.loc[:,c].shift(i)
    
    df_shift.dropna(inplace=True)
    
    return df_shift

## 1. Train 전처리

In [ ]:
def train_processing(df):
    train = df[df["IDX"]!=777]
    
    train_shift = shifting(train)
    ts_idx = list(train_shift.index)
    
    drop_idx = []
    for i in ts_idx:
        idx_num = train_shift.loc[i,'IDX']
        if idx_num <= train_shift.loc[i,'shift_IDX']:
            drop_idx.append(i)
    

    train_s_df = train_shift.drop(drop_idx).reset_index(drop = True)  
    train_s_drop_df = train_s_df.drop(["IDX","MERGE_IDX","shift_IDX","shift_MERGE_IDX","ERA","AVG","PCT"],axis = 1)

    return train_s_df, train_s_drop_df

In [ ]:
orig_train1 = []
orig_train2 = []

for i in range(len(orig_data)):
    tmp_df = orig_data[i]
    train_s_df, train_s_drop_df = train_processing(tmp_df)
    orig_train1.append(train_s_df)
    orig_train2.append(train_s_drop_df)

In [ ]:
cumul_train1 = []
cumul_train2 = []

for i in range(len(new_cumul_data)):    
    tmp_df_c = new_cumul_data[i]
    train_s_df_c, train_s_drop_df_c = train_processing(tmp_df_c)
    cumul_train1.append(train_s_df_c)
    cumul_train2.append(train_s_drop_df_c)

## 2. Valid 전처리

In [ ]:
def valid_processing(df, train_year):
    test = df[df["IDX"]==777]
    tmp_valid = df[df["IDX"]==5]
    
    tmp_valid = tmp_valid[(tmp_valid["YEAR"] != str(train_year[0])) & (tmp_valid["YEAR"] !=str(train_year[1]))]
    
    test = pd.concat([test,tmp_valid],axis=0)
    test = test.sort_values(by=["YEAR","T_ID","IDX"]).reset_index(drop=True)

    test_shift = shifting(test)

    test_s_df = test_shift[test_shift["IDX"]==777].reset_index(drop=True)
    test_s_drop_df = test_s_df.drop(["IDX","MERGE_IDX","shift_IDX","shift_MERGE_IDX","ERA","AVG","PCT"],axis = 1)
    
    return test_s_df, test_s_drop_df

In [ ]:
orig_valid1 = []
orig_valid2 = []

for i in range(len(orig_data)):
    tmp_df = orig_data[i]
    valid_s_df, valid_s_drop_df = valid_processing(tmp_df,[i+2016,2020])
    orig_valid1.append(valid_s_df)
    orig_valid2.append(valid_s_drop_df)

In [ ]:
cumul_valid1 = []
cumul_valid2 = []

for i in range(len(new_cumul_data)):    
    tmp_df_c = new_cumul_data[i]
    valid_s_df_c, valid_s_drop_df_c = valid_processing(tmp_df_c,[i+2016,2020])
    cumul_valid1.append(valid_s_df_c)
    cumul_valid2.append(valid_s_drop_df_c)

## 3. AVG/ERA dataset 가공

In [ ]:

def select_col(train_col):
    common_list = ['T_ID', 'YEAR', 'HEADER_NO','PCT']
    hitter = ['AVG', 'OBP', 'SLG', 'OPS', 'RC', 'GPA', 'BA', 'SECA', 
              'TA', 'XR', 'ISO', 'wOBA', 'P_HRA_RT', 'DER', 'a_']
    pitcher = ['WHIP', 'LOB', 'ERA', 'FIP', 'H_9', 'K_9', 'BB_9', 
               'HR_9', 'oAVG', 'oSLG', 'oOBP', 'P_WHIP_RT', 'P2_WHIP_RT', 'CB_WHIP_RT','oISO', 'oOPS','e_']

    vs_hitter = ['vs_AVG', 'vs_OBP', 'vs_SLG', 'vs_OPS', 'vs_RC', 'vs_GPA', 'vs_BA', 'vs_SECA', 
                 'vs_TA', 'vs_XR', 'vs_ISO', 'vs_wOBA', 'vs_P_HRA_RT', 'vs_DER']
    vs_pitcher = ['vs_WHIP', 'vs_LOB', 'vs_ERA', 'vs_FIP', 'vs_H_9', 'vs_K_9', 'vs_BB_9', 
                  'vs_HR_9', 'vs_oAVG', 'vs_oSLG', 'vs_oOBP', 'vs_P_WHIP_RT', 'vs_P2_WHIP_RT', 'vs_CB_WHIP_RT'
                 ,'vs_oISO', 'vs_oOPS']

    avg_col = common_list + hitter + vs_pitcher
    era_col = common_list + pitcher + vs_hitter

    sel_avg_lst =train_col[:2]
    sel_era_lst = train_col[:2]

    for i in train_col[2:]:
        for k in avg_col:
            if k in i:
                sel_avg_lst.append(i)
                break

    for i in train_col[2:]:
        for k in era_col:
            if k in i:
                sel_era_lst.append(i)
                break
                
    return sel_avg_lst, sel_era_lst

In [ ]:
train_col = list(cumul_train2[0].columns)
sel_avg_lst, sel_era_lst = select_col(train_col)

## 4. 최종 데이터셋 생성 및 저장

### 4-1. 생성(Train, Valid)

In [ ]:
hvs_col = ['c_HH', 'c_HT', 'c_KT', 'c_LG', 'c_LT', 'c_NC', 'c_OB', 'c_SK', 'c_SS', 'c_WO',"ISHOME"]

In [ ]:
# 승률
PCT_train_X_lst = []
PCT_train_y_lst = []

PCT_valid_X_lst = []
PCT_valid_y_lst = []

for i in range(len(orig_train1)):
    hvs_tmp = Home_VS_lst[i]
    tX = hvs_tmp[(hvs_tmp["IDX"] !=777)&(hvs_tmp["IDX"] !=1)].loc[:,hvs_col].reset_index(drop=True)
    vX = hvs_tmp[hvs_tmp["IDX"]==777].loc[:,hvs_col].reset_index(drop=True)

    PCT_train_X = orig_train2[i]
    PCT_train_y = orig_train1[i][["T_ID","YEAR","PCT"]]
    
    PCT_valid_X = orig_valid2[i]
    PCT_valid_y = orig_valid1[i][["T_ID","YEAR","PCT"]]
    
    PCT_train_X_lst.append(pd.concat([PCT_train_X,tX],axis=1))
    PCT_train_y_lst.append(PCT_train_y)
    
    PCT_valid_X_lst.append(pd.concat([PCT_valid_X,vX],axis=1))
    PCT_valid_y_lst.append(PCT_valid_y)

In [ ]:
# 타율
AVG_train_X_lst = []
AVG_train_y_lst = []

AVG_valid_X_lst = []
AVG_valid_y_lst = []

for i in range(len(cumul_train1)):
    hvs_tmp = Home_VS_lst[i]
    tX = hvs_tmp[(hvs_tmp["IDX"] !=777)&(hvs_tmp["IDX"] !=1)].loc[:,hvs_col].reset_index(drop=True)
    vX = hvs_tmp[hvs_tmp["IDX"]==777].loc[:,hvs_col].reset_index(drop=True)

    AVG_train_X = cumul_train2[i][sel_avg_lst]
    AVG_train_y = orig_train1[i][["T_ID","YEAR","AVG"]]
    
    AVG_valid_X = cumul_valid2[i][sel_avg_lst]
    AVG_valid_y = orig_valid1[i][["T_ID","YEAR","AVG"]]
    
    AVG_train_X_lst.append(pd.concat([AVG_train_X,tX],axis=1))
    AVG_train_y_lst.append(AVG_train_y)
    
    AVG_valid_X_lst.append(pd.concat([AVG_valid_X,vX],axis=1))
    AVG_valid_y_lst.append(AVG_valid_y)

In [ ]:
# 방어율
ERA_train_X_lst = []
ERA_train_y_lst = []

ERA_valid_X_lst = []
ERA_valid_y_lst = []

for i in range(len(cumul_train1)):
    hvs_tmp = Home_VS_lst[i]
    tX = hvs_tmp[(hvs_tmp["IDX"] !=777)&(hvs_tmp["IDX"] !=1)].loc[:,hvs_col].reset_index(drop=True)
    vX = hvs_tmp[hvs_tmp["IDX"]==777].loc[:,hvs_col].reset_index(drop=True)

    ERA_train_X = cumul_train2[i][sel_era_lst]
    ERA_train_y = orig_train1[i][["T_ID","YEAR","ERA"]]
    
    ERA_valid_X = cumul_valid2[i][sel_era_lst]
    ERA_valid_y = orig_valid1[i][["T_ID","YEAR","ERA"]]
    
    ERA_train_X_lst.append(pd.concat([ERA_train_X,tX],axis=1))
    ERA_train_y_lst.append(ERA_train_y)
    
    ERA_valid_X_lst.append(pd.concat([ERA_valid_X,vX],axis=1))
    ERA_valid_y_lst.append(ERA_valid_y)

In [ ]:
ERA_train_X.shape, ERA_train_y.shape, ERA_valid_X.shape, ERA_valid_y.shape

In [ ]:
AVG_train_X.shape, AVG_train_y.shape, AVG_valid_X.shape, AVG_valid_y.shape

In [ ]:
PCT_train_X.shape, PCT_train_y.shape, PCT_valid_X.shape, PCT_valid_y.shape

### 4-2. 생성(Test)

In [ ]:
# PCT, AVG,ERA 따로

size_2020_df = size_df[size_df["YEAR"]=='2020'].drop(["GDAY_DS"],axis=1)
sum_2020_df = sum_df[sum_df["YEAR"]=='2020'].drop(["GDAY_DS"],axis=1)

test_orig_df = pd.DataFrame([])
test_cumul_df = pd.DataFrame([])

key = 24
for t in team:
    ttmp_size_df = size_2020_df[size_2020_df["T_ID"]==t]
    ttmp_sum_df = sum_2020_df[sum_2020_df["T_ID"]==t]
    tmp_size = len(ttmp_size_df)
    
    # 누적 X
    ttmp_size_test_df = ttmp_size_df.iloc[tmp_size-key:,:]
    ttmp_sum_test_df = ttmp_sum_df.iloc[tmp_size-key:,:]
    
    tmp1 = make_pct(ttmp_size_test_df)[["PCT"]]
    tmp2 = make_sum(ttmp_sum_test_df)
    
    merge_df1 = pd.concat([tmp2,tmp1], axis=1)
    conv1 = convert_attribute(merge_df1, 'test')
    
    test_orig_df = pd.concat([test_orig_df,conv1],axis=0)
    
    # 누적 O
    ttmp_size_test_df_c = ttmp_size_df.iloc[:tmp_size-key,:]
    ttmp_sum_test_df_c = ttmp_sum_df.iloc[:tmp_size-key,:]
    
    tmp1_c = make_pct(ttmp_size_test_df_c)[["PCT"]]
    tmp2_c = make_sum(ttmp_sum_test_df_c)
    
    merge_df2 = pd.concat([tmp2_c,tmp1_c], axis=1)
    conv2 = convert_attribute(merge_df2, 'test')
    
    test_cumul_df = pd.concat([test_cumul_df,conv2],axis=0)

test_orig_df = test_orig_df.reset_index(drop=True)  
test_cumul_df = test_cumul_df.reset_index(drop=True)
    
# 누적 승타방
ae_team_df = sum_2020_df[def_col + ae_col]
ae_team_df_c = make_sum(ae_team_df, 'vs')

ae_team_df_c["ERA"] = (9 * ae_team_df_c["p_ER"]) / (ae_team_df_c["p_INN2"] / 3)
ae_team_df_c["AVG"] = (ae_team_df_c["h_HIT"] / ae_team_df_c["h_AB"])

cul_t_avg_df_c = ae_team_df_c.drop(["ERA"]+ae_col,axis=1)
cul_t_era_df_c = ae_team_df_c.drop(["AVG"]+ae_col,axis=1)

cul_avg_df_c = cul_t_avg_df_c.pivot_table('AVG', ['T_ID', 'YEAR'], 'VS_T_ID').reset_index().fillna(0)
cul_era_df_c = cul_t_era_df_c.pivot_table('ERA', ['T_ID', 'YEAR'], 'VS_T_ID').reset_index().fillna(0)

ca_c = cul_avg_df_c.iloc[:,-10:].add_prefix('a_')
ce_c = cul_era_df_c.iloc[:,-10:].add_prefix('e_')

test_cumul_df_c = pd.concat([test_cumul_df,ca_c,ce_c],axis=1)


test_col = list(test_cumul_df_c.columns)
sel_avg_lst_t, sel_era_lst_t = select_col(test_col)

PCT_test_X = test_orig_df.copy()
AVG_test_X = test_cumul_df_c[sel_avg_lst_t]
ERA_test_X = test_cumul_df_c[sel_era_lst_t]
    
# 이번에 제공된 test로 구성
# df_new = pd.concat([df, pd.get_dummies(df.VS_T_ID)], axis = 1)

# for i in range(len(df_new)):
#     if df_new.loc[i,'T_ID'] == df_new.loc[i,"G_ID"][10:12]:
#         df_new.loc[i,'ISHOME'] = 1
#     else:
#         df_new.loc[i,'ISHOME'] = 0
        
# df_new["ISHOME"] = df_new["ISHOME"].astype(np.int64)

# home_n_vs_df = df_new[["G_ID","YEAR","T_ID","VS_T_ID","ISHOME"]+team]


In [ ]:
PCT_train_X_lst[0].shape, AVG_train_X_lst[0].shape,ERA_train_X_lst[0].shape

In [ ]:
PCT_test_X.shape, AVG_test_X.shape, ERA_test_X.shape

### 4-2. 저장

In [ ]:
for i in range(len(PCT_train_X_lst)):
    PCT_train_X_lst[i].to_csv("../data/PCT/PCT_train_X_{}.csv".format(i+1),index = False)
    PCT_train_y_lst[i].to_csv("../data/PCT/PCT_train_y_{}.csv".format(i+1),index = False)

    PCT_valid_X_lst[i].to_csv("../data/PCT/PCT_valid_X_{}.csv".format(i+1),index = False)
    PCT_valid_y_lst[i].to_csv("../data/PCT/PCT_valid_y_{}.csv".format(i+1),index = False)

    AVG_train_X_lst[i].to_csv("../data/AVG/AVG_train_X_{}.csv".format(i+1),index = False)
    AVG_train_y_lst[i].to_csv("../data/AVG/AVG_train_y_{}.csv".format(i+1),index = False)

    AVG_valid_X_lst[i].to_csv("../data/AVG/AVG_valid_X_{}.csv".format(i+1),index = False)
    AVG_valid_y_lst[i].to_csv("../data/AVG/AVG_valid_y_{}.csv".format(i+1),index = False)
    
    ERA_train_X_lst[i].to_csv("../data/ERA/ERA_train_X_{}.csv".format(i+1),index = False)
    ERA_train_y_lst[i].to_csv("../data/ERA/ERA_train_y_{}.csv".format(i+1),index = False)

    ERA_valid_X_lst[i].to_csv("../data/ERA/ERA_valid_X_{}.csv".format(i+1),index = False)
    ERA_valid_y_lst[i].to_csv("../data/ERA/ERA_valid_y_{}.csv".format(i+1),index = False)
